In [1]:
# @title Installations
# The %%capture command hides the installation output.
%%capture
!pip install langchain
!pip install lark
!pip install openai
!pip install chromadb
!pip install langchain_openai
!pip install langchain_chroma
!pip install langchain_community
!pip install pypdf

In [16]:
# @title Imports
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
import os
from google.colab import userdata
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.retrievers.multi_query import MultiQueryRetriever
import logging

In [7]:
# @title Docs
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf")
pdf_data = loader.load()
pdf_data[1]

Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2'}, page_content='LangChain helps us to unlock the ability to harness the \nLLM’s immense potential in tasks such as document analysis, \nchatbot development, code analysis, and countless other \napplications. Whether your desire is to unlock deeper natural \nlanguage understanding , enhance data, or circumvent \nlanguage barriers through translation, LangChain is ready to \nprovide the tools and programming support you need to do \nwithout it that it is not only difficult but also fresh for you. Its \ncore functionalities encompass: \n1. Context-Aware Capabilities: LangChain facilitates the \ndevelopment of applications that ar

In [10]:
# @title Environment Setup
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [12]:
# @title Splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
chunks_pdf = text_splitter.split_documents(pdf_data)

# VectorDB
vectordb = Chroma(embedding_function=OpenAIEmbeddings()) # Initialize vectordb
vectordb = Chroma.from_documents(documents=chunks_pdf, embedding=OpenAIEmbeddings())

In [15]:
# @title Query
query = "What does the paper say about langchain?"

llm = ChatOpenAI(temperature=0) # Initialize the llm

retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

/tmp/ipython-input-2315263131.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0) # Initialize the llm


In [17]:
# @title logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [18]:
docs = retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on langchain as discussed in the paper?', '2. What are the key points mentioned about langchain in the paper?', '3. How is langchain described or analyzed in the paper?']


[Document(metadata={'author': 'IEEE', 'title': 's8329 final', 'moddate': '2023-12-31T03:52:06+00:00', 'total_pages': 6, 'creator': 'Microsoft Word', 'page_label': '1', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ioch1wsxkfqgfLLgmd-6Rw/langchain-paper.pdf', 'creationdate': '2023-12-31T03:50:13+00:00', 'page': 0, 'producer': 'PyPDF'}, page_content='II. LANGCHAIN \nLangChain, with its open -source essence, emerges as a \npromising solution, aiming to simplify the complex process of \ndeveloping applications powered by large language models \n(LLMs). This framework though the rapid delivery of building \nblocks and pre-built chains for building large language model \napplications shows the easy way developers can do it.'),
 Document(metadata={'page_label': '2', 'author': 'IEEE', 'title': 's8329 final', 'page': 1, 'total_pages': 6, 'producer': 'PyPDF', 'creationdate': '2023-12-31T03:50:13+00:00', 'moddate': '2023-12-31T03:52:06+00:00', 'source': 'https://cf